In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')




### Wheeze_188 Distribution

In [9]:
Total = np.unique(pd.read_csv("./188_wheeze.txt", header=None)[0].values)
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [14]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 188
Asthmatic_male = 60
Asthmatic_female = 46
Healthy_Male = 42
Healthy_female = 34
Before_after = 6


### Breathe

In [16]:
Total = np.unique(os.listdir("./breathe_mfcc/"))
name = []
for i in Total:
    name.append(i[:4])

unique = []
before_after = []
for i in Total:
    if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
        unique.append(i)
    else:
        before_after.append(i) 

Asthmatic_Female = []
Asthmatic_Male = []
Healthy_Male = []
Healthy_Female = []
for file in unique:
    if file.find('sthma') !=-1:
        if file.find("_M_")!=-1:
            Asthmatic_Male.append(file)
        if file.find("_F_")!=-1:
            Asthmatic_Female.append(file)
    if file.find("_C_")!=-1:
        if file.find("_M_")!=-1:
            Healthy_Male.append(file)
        if file.find("_F_")!=-1:
            Healthy_Female.append(file)

In [17]:
print(f"Total = {len(Total)}")
print(f"Asthmatic_male = {len(Asthmatic_Male)}")
print(f"Asthmatic_female = {len(Asthmatic_Female)}")
print(f"Healthy_Male = {len(Healthy_Male)}")
print(f"Healthy_female = {len(Healthy_Female)}")
print(f"Before_after = {len(before_after)}")

Total = 258
Asthmatic_male = 78
Asthmatic_female = 55
Healthy_Male = 55
Healthy_female = 50
Before_after = 20


#### Helper functions

In [18]:
def train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after):
    random.seed(0)
    test_set = np.concatenate(((np.random.choice(np.array(Asthmatic_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Asthmatic_Female), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Male), replace=False, size=(7)),
                                np.random.choice(np.array(Healthy_Female), replace=False, size=(7)))))
    train_set = np.concatenate((np.array(Asthmatic_Male),
                                np.array(Asthmatic_Female),
                                np.array(Healthy_Male),
                                np.array(Healthy_Female)))
    train_set = np.setdiff1d(train_set,test_set)
#     np.array(before_after).shape
    train_set = np.concatenate((train_set, np.array(before_after)))
    return train_set, test_set

In [19]:
# train_test Csv
def train_test_csv(sound_dataframe):
    Total = np.unique(sound_dataframe.iloc[:,sound_dataframe.shape[1]-1])
    name = []
    for i in Total:
        name.append(i[:4])

    unique = []
    before_after = []
    for i in Total:
        if np.sum((np.array(name, dtype = int) == int(i[:4]))) == 1:
            unique.append(i)
        else:
            before_after.append(i) 

    Asthmatic_Female = []
    Asthmatic_Male = []
    Healthy_Male = []
    Healthy_Female = []
    for file in unique:
#         if file.find('sthma') !=-1:
            if file.find("_M_")!=-1:
                Asthmatic_Male.append(file)
            if file.find("_F_")!=-1:
                Asthmatic_Female.append(file)
        if file.find("_C_")!=-1:
            if file.find("_M_")!=-1:
                Healthy_Male.append(file)
            if file.find("_F_")!=-1:
                Healthy_Female.append(file)

                

    Train, Test = train_test_set(Asthmatic_Female, Asthmatic_Male, Healthy_Female, Healthy_Male, before_after)     

    test_dataframe = pd.DataFrame()
    for i, line in enumerate(Test):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[i]))]
        test_dataframe = pd.DataFrame.append(test_dataframe,A)
        
        
    train_dataframe = pd.DataFrame()
    for i, line in enumerate(Train):
        A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[i]))]
        train_dataframe = pd.DataFrame.append(train_dataframe,A) 
        
        
    
    return train_dataframe, test_dataframe

In [2]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(sound_dataframe, sound, path):    
    MLA = [
        #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),

        #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),

        #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),

        #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(probability=True),
#         svm.NuSVC(probability=True),
        svm.LinearSVC(),

        #Trees    
#         tree.DecisionTreeClassifier(),
# #         tree.ExtraTreeClassifier(),

        #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier(max_depth = 8,
                     subsample = 0.8,
                     learning_rate = 0.01,
                     n_estimators = 450,
                     min_child_weight = 1)
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,10)):
            train_csv, test_csv = pd.read_csv(path+'/train'+sound+str(i)+'.csv'), pd.read_csv(path+'./test'+sound+str(i)+'.csv')
            X_train = train_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_train = train_csv.iloc[:,sound_dataframe.shape[1]-2]
            X_test = test_csv.iloc[:,:sound_dataframe.shape[1]-2]
            y_test = test_csv.iloc[:,sound_dataframe.shape[1]-2]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            
#             if alg.__class__.__name__ == 'XGBClassifier':
#                 filename = './CMN/XGB_model'+ str(i) + '.model'
#                 pickle.dump(alg, open(filename, 'wb'))
#                 train_csv.to_csv('./CMN/Train_XGB'+str(i)+'.csv', index=False)
                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [47]:
# # Making Sets
# def make_and_save_sets(sound_dataframe, sound):
#     for i in range(0,10):
#         train_csv, test_csv = train_test_csv(sound_dataframe)
#         train_csv.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
#         test_csv.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)

def make_legit_sets(sound_dataframe, sound):
    for i in range(0,10):
        Train = np.loadtxt(f"./Wheeze_sets/Train/Train_{i}.txt", dtype= 'str')
        Test = np.loadtxt(f"./Wheeze_sets/Test/Test_{i}.txt", dtype= 'str')


        test_dataframe = pd.DataFrame()
        for j, line in enumerate(Test):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Test[j]))]
            test_dataframe = pd.DataFrame.append(test_dataframe,A)

        train_dataframe = pd.DataFrame()
        for j, line in enumerate(Train):
            A = sound_dataframe[(sound_dataframe[str(sound_dataframe.shape[1]-1)] == (Train[j]))]
            train_dataframe = pd.DataFrame.append(train_dataframe,A)

        train_dataframe.to_csv('./breathe_set_csv/train'+sound+str(i)+'.csv', index=False)
        test_dataframe.to_csv('./breathe_set_csv/test'+sound+str(i)+'.csv', index=False)
    return None

In [7]:
Wheeze_dataframe = pd.read_csv("./MFCCs_stasts_Csv/Wheeze.csv")
Breath_dataframe = pd.read_csv("./breath_csv/Breath.csv")

In [33]:
make_and_save_sets(Wheeze_dataframe, 'Wheeze')

In [36]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'wheeze', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[15:30:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:06<00:58,  6.50s/it]

[15:31:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:10<00:41,  5.13s/it]

...

 80%|████████  | 8/10 [00:33<00:07,  3.83s/it]

[15:31:27] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:36<00:03,  3.68s/it]

[15:31:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:40<00:00,  4.00s/it]


In [37]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9920863309352518, 0.9934306569343065, 0.991...",0.991533,"[0.8968609865470852, 0.8641975308641975, 0.751...",0.860558,0.0612982
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 1.0, 0.9992690058479532, ...",0.999562,"[0.8923766816143498, 0.8683127572016461, 0.767...",0.859775,0.0552414
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8834080717488789, 0.8600823045267489, 0.763...",0.858964,0.0582834
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9784172661870504, 0.9773722627737226, 0.978...",0.97459,"[0.8609865470852018, 0.831275720164609, 0.7632...",0.82091,0.040913
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8985611510791367, 0.9138686131386862, 0.921...",0.902892,"[0.8295964125560538, 0.7366255144032922, 0.706...",0.794729,0.0560749


In [37]:
# make_and_save_sets(Breath_dataframe, 'Breath')

In [38]:
# MLA_Breath = MLA_selection(Breath_dataframe, 'Breath', "./Set_CSV/")

  0%|          | 0/10 [00:00<?, ?it/s]

[10:34:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:00<00:08,  1.12it/s]

[10:34:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:01<00:07,  1.14it/s]

...

 80%|████████  | 8/10 [00:07<00:01,  1.06it/s]

[10:35:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:08<00:00,  1.07it/s]

[10:35:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [49]:
Wheeze_dataframe = pd.read_csv("./breath_csv/Wheeze.csv")
Wheeze_dataframe_1 = pd.read_csv("./breath_csv/Wheeze_1.csv")
Wheeze_dataframe_2 = pd.read_csv("./breath_csv/Wheeze_2.csv")
Wheeze_dataframe_3 = pd.read_csv("./breath_csv/Wheeze_3.csv")

In [50]:
make_legit_sets(Wheeze_dataframe, 'Wheeze')
make_legit_sets(Wheeze_dataframe_1, 'Wheeze_1')
make_legit_sets(Wheeze_dataframe_2, 'Wheeze_2')
make_legit_sets(Wheeze_dataframe_3, 'Wheeze_3')

In [51]:
MLA_Wheeze = MLA_selection(Wheeze_dataframe, 'Wheeze', "./breathe_set_csv/")
MLA_Wheeze_1 = MLA_selection(Wheeze_dataframe_1, 'Wheeze_1', "./breathe_set_csv/")
MLA_Wheeze_2 = MLA_selection(Wheeze_dataframe_2, 'Wheeze_2', "./breathe_set_csv/")
MLA_Wheeze_3 = MLA_selection(Wheeze_dataframe_3, 'Wheeze_3', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[15:41:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:03<00:30,  3.43s/it]

[15:41:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:06<00:26,  3.32s/it]

...

 80%|████████  | 8/10 [00:25<00:06,  3.24s/it]

[15:45:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [00:29<00:03,  3.25s/it]

[15:45:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [00:32<00:00,  3.23s/it]


In [52]:
MLA_Wheeze

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 1.0, 0.9992690058479532, ...",0.99971,"[0.8923766816143498, 0.8683127572016461, 0.759...",0.859796,0.0572516
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8923766816143498, 0.8559670781893004, 0.751...",0.857264,0.0582195
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9935251798561151, 0.9927007299270073, 0.991...",0.991896,"[0.8923766816143498, 0.8641975308641975, 0.726...",0.850744,0.0645768
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9784172661870504, 0.9766423357664233, 0.979...",0.974519,"[0.852017937219731, 0.8271604938271605, 0.7428...",0.817573,0.0435298
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8985611510791367, 0.910948905109489, 0.9210...",0.902301,"[0.8295964125560538, 0.7366255144032922, 0.702...",0.793896,0.0562414


In [53]:
MLA_Wheeze_1

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8340807174887892, 0.8148148148148148, 0.661...",0.789876,0.0796238
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9978417266187051, 1.0, 0.9985380116959064, ...",0.999565,"[0.8295964125560538, 0.7860082304526749, 0.665...",0.783063,0.0749197
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9741007194244604, 0.9766423357664233, 0.981...",0.97132,"[0.8026905829596412, 0.7695473251028807, 0.644...",0.775536,0.0743698
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9410071942446043, 0.9437956204379562, 0.952...",0.937517,"[0.8071748878923767, 0.7613168724279835, 0.673...",0.774306,0.0727206
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8597122302158273, 0.8751824817518248, 0.889...",0.867122,"[0.8161434977578476, 0.7366255144032922, 0.693...",0.767092,0.0588716


In [54]:
MLA_Wheeze_2

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.831275720164609, 0.73877...",0.838803,0.0485347
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8834080717488789, 0.8395061728395061, 0.689...",0.836816,0.063978
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9913669064748202, 0.9905109489051095, 0.989...",0.989931,"[0.8878923766816144, 0.8353909465020576, 0.702...",0.831058,0.0622819
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9683453237410072, 0.9700729927007299, 0.974...",0.967453,"[0.8834080717488789, 0.8436213991769548, 0.714...",0.827548,0.0572437
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.89568345323741, 0.9, 0.9166666666666666, 0....",0.894649,"[0.8475336322869955, 0.7407407407407407, 0.673...",0.791801,0.0600646


In [55]:
MLA_Wheeze_3

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9517985611510792, 0.9518248175182482, 0.959...",0.951178,"[0.8475336322869955, 0.8024691358024691, 0.742...",0.795867,0.0364929
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8733812949640288, 0.8868613138686131, 0.907...",0.882832,"[0.8385650224215246, 0.7736625514403292, 0.681...",0.791925,0.0662885
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8565022421524664, 0.7695473251028807, 0.673...",0.789259,0.0538737
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9992805755395684, 0.9992700729927008, 0.999...",0.99949,"[0.852017937219731, 0.7736625514403292, 0.6530...",0.782801,0.0545316
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9669064748201439, 0.9773722627737226, 0.977...",0.973293,"[0.8609865470852018, 0.7654320987654321, 0.657...",0.7775,0.0546414


In [56]:
Wheeze_dataframe_0 = pd.read_csv("./breath_csv/Wheeze_0.csv")
Wheeze_dataframe_5 = pd.read_csv("./breath_csv/Wheeze_5.csv")
Wheeze_dataframe_10 = pd.read_csv("./breath_csv/Wheeze_10.csv")
Wheeze_dataframe_15 = pd.read_csv("./breath_csv/Wheeze_15.csv")
Wheeze_dataframe_20 = pd.read_csv("./breath_csv/Wheeze_20.csv")
Wheeze_dataframe_25 = pd.read_csv("./breath_csv/Wheeze_25.csv")
Wheeze_dataframe_30 = pd.read_csv("./breath_csv/Wheeze_30.csv")
Wheeze_dataframe_35 = pd.read_csv("./breath_csv/Wheeze_35.csv")
Wheeze_dataframe_40 = pd.read_csv("./breath_csv/Wheeze_40.csv")
Wheeze_dataframe_45 = pd.read_csv("./breath_csv/Wheeze_45.csv")
Wheeze_dataframe_50 = pd.read_csv("./breath_csv/Wheeze_50.csv")


In [57]:
make_legit_sets(Wheeze_dataframe_0, 'Wheeze_0')
make_legit_sets(Wheeze_dataframe_5, 'Wheeze_5')
make_legit_sets(Wheeze_dataframe_10, 'Wheeze_10')
make_legit_sets(Wheeze_dataframe_15, 'Wheeze_15')
make_legit_sets(Wheeze_dataframe_20, 'Wheeze_20')
make_legit_sets(Wheeze_dataframe_25, 'Wheeze_25')
make_legit_sets(Wheeze_dataframe_30, 'Wheeze_30')
make_legit_sets(Wheeze_dataframe_35, 'Wheeze_35')
make_legit_sets(Wheeze_dataframe_40, 'Wheeze_40')
make_legit_sets(Wheeze_dataframe_45, 'Wheeze_45')
make_legit_sets(Wheeze_dataframe_50, 'Wheeze_50')

In [58]:
MLA_wheeze_0 = MLA_selection(Wheeze_dataframe_0, 'Wheeze_0', "./breathe_set_csv/")
MLA_wheeze_5 = MLA_selection(Wheeze_dataframe_5, 'Wheeze_5', "./breathe_set_csv/")
MLA_wheeze_10 = MLA_selection(Wheeze_dataframe_10, 'Wheeze_10', "./breathe_set_csv/")
MLA_wheeze_15 = MLA_selection(Wheeze_dataframe_15, 'Wheeze_15', "./breathe_set_csv/")
MLA_wheeze_20 = MLA_selection(Wheeze_dataframe_20, 'Wheeze_20', "./breathe_set_csv/")
MLA_wheeze_25 = MLA_selection(Wheeze_dataframe_25, 'Wheeze_25', "./breathe_set_csv/")
MLA_wheeze_30 = MLA_selection(Wheeze_dataframe_30, 'Wheeze_30', "./breathe_set_csv/")
MLA_wheeze_35 = MLA_selection(Wheeze_dataframe_35, 'Wheeze_35', "./breathe_set_csv/")
MLA_wheeze_40 = MLA_selection(Wheeze_dataframe_40, 'Wheeze_40', "./breathe_set_csv/")
MLA_wheeze_45 = MLA_selection(Wheeze_dataframe_45, 'Wheeze_45', "./breathe_set_csv/")
MLA_wheeze_50 = MLA_selection(Wheeze_dataframe_50, 'Wheeze_50', "./breathe_set_csv/")

  0%|          | 0/10 [00:00<?, ?it/s]

[15:47:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 10%|█         | 1/10 [00:03<00:28,  3.20s/it]

[15:47:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 2/10 [00:06<00:24,  3.12s/it]

...

 80%|████████  | 8/10 [01:03<00:12,  6.46s/it]

[16:03:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 90%|█████████ | 9/10 [01:10<00:06,  6.68s/it]

[16:04:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 10/10 [01:16<00:00,  7.69s/it]


In [60]:
MLA_wheeze_0

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8699551569506726, 0.8271604938271605, 0.706...",0.827852,0.0733743
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[0.9985611510791367, 1.0, 0.9985380116959064, ...",0.999417,"[0.8789237668161435, 0.7942386831275721, 0.681...",0.813664,0.0723899
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9856115107913669, 0.9875912408759124, 0.991...",0.985689,"[0.8475336322869955, 0.7901234567901234, 0.693...",0.812331,0.0665325
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9568345323741008, 0.9525547445255474, 0.963...",0.952998,"[0.852017937219731, 0.8024691358024691, 0.6775...",0.804584,0.0695436
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8877697841726618, 0.891970802919708, 0.9071...",0.888505,"[0.852017937219731, 0.7818930041152263, 0.7061...",0.797002,0.0684282


In [61]:
MLA_wheeze_25

,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
4,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.874439461883408, 0.831275720164609, 0.73877...",0.838803,0.0485347
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1,"[0.8699551569506726, 0.831275720164609, 0.7102...",0.834474,0.0544568
0,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9913669064748202, 0.9905109489051095, 0.989...",0.989931,"[0.8878923766816144, 0.8353909465020576, 0.702...",0.831475,0.0628644
2,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.9683453237410072, 0.9700729927007299, 0.974...",0.967453,"[0.8834080717488789, 0.8436213991769548, 0.714...",0.827548,0.0572437
3,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.8949640287769784, 0.9007299270072993, 0.916...",0.89385,"[0.8385650224215246, 0.7448559670781894, 0.673...",0.791338,0.0586144
